<a href="https://colab.research.google.com/github/qkrqhdud/GMB_Gemma2-/blob/main/PPP/Gradio_PPP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install streamlit
!pip install torch
!pip install unsloth
!pip install bitsandbytes
!pip install transformers
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 83.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.3/79.3 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.0/56.0 kB 2.3 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.7/165.7 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.5/322.5 kB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.4/209.4 MB

In [8]:
from transformers import pipeline
from unsloth import FastLanguageModel
import re

def load_model():
    finetune_model,tokenizer = FastLanguageModel.from_pretrained(
    model_name = "yunho123/lora_model", # YOUR MODEL YOU USED FOR TRAINING
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True,
    )
    FastLanguageModel.for_inference(finetune_model)
    return finetune_model,tokenizer

def get_output(post_input,finetune_model,tokenizer):
  # 프롬프트
  post_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the requegr.

  ### Input:
  {}

  ### Instruction:
  "Please rate the following blog post from 1 to 10 on each of the following category and provide a brief comment explaining your score for each criterion: "Topic Relevance", "Value Provision", "Storytelling", "Expertise","Conciseness","Paragraph Structure","Attention-Grabbing Opening", "Call-to-Action (CTA)","Balance of Professionalism and Personal Touch" and "Hashtag Usage"
   Please provide the output in the following format."

    "category": "Topic Relevance",
    "score": 5,
    "explanation": "comment",
    "improvement_suggestion": "comment"
    #next category
    "category": ,
    "score": ,
    "explanation": "comment",
    "improvement_suggestion": "comment"


  ### Response:
  {}"""
  inputs = tokenizer(
  [
      post_prompt.format(
          post_input,
          ""
      )
  ], return_tensors = "pt").to("cuda")

  outputs = tokenizer.batch_decode(finetune_model.generate(**inputs, max_new_tokens = 516, use_cache = True))

  response_start = outputs[0].find("### Response:")
  if response_start != -1:
    output = outputs[0][response_start+len("### Response:"):].strip()

  result = []
  matches = re.findall(r'"category":\s*"(.*?)",\s*\n\s*"score":\s*(.*?),\s*\n\s*"explanation":\s*"(.*?)",\s*\n\s*"improvement_suggestion":\s*"(.*?)"', output)

  for match in matches:
      category, score, explanation, improvement_suggestion = match
      result.append({"category": category, "score": int(score), "explanation": explanation, "improvement_suggestion": improvement_suggestion})


  # 결과 출력
  return result

# TEST DATA post_input = 'Agree with  Zach Boren , Johns post highlights the importance of finding opportunities for young people  #apprenticeshipworks'

In [10]:
import numpy as np
import plotly.graph_objects as go
import gradio as gr

model,tokenizer = load_model()

# 별점 계산을 위한 함수
def get_star_rating(score):
    full_stars = int(score)
    half_star = score - full_stars
    return "★" * full_stars + ("½" if half_star >= 0.5 else "") + "☆" * (5 - full_stars - (1 if half_star >= 0.5 else 0))

# 박스 형태 점수 표시 함수
def score_box(score):
    filled = "■" * score
    empty = "□" * (10 - score)
    return f"{filled}{empty}"

# 분석 버튼이 클릭될 때 수행할 작업
def analyze_post(post_input, tone, purpose, engagement):

    if post_input.strip() == "":
        return "게시물을 입력해주세요.", None, [], "", "", "", ""
    else:
        results = get_output(post_input, model, tokenizer)
        user_score_average = np.mean(results["scores"])
        #dummy_average = 5.8  # 더미 평균

        user_star_rating = get_star_rating(user_score_average / 2)
        #dummy_star_rating = get_star_rating(dummy_average / 2)

        # Plotly 레이더 차트 생성
        fig = go.Figure()
        fig.add_trace(go.Scatterpolar(
            r=results["scores"] + [results["scores"][0]],
            theta=results["categories"] + [results["categories"][0]],
            fill='toself',
            line=dict(color='blue')
        ))
        fig.update_layout(
            polar=dict(
                radialaxis=dict(visible=True, range=[0, 10])
            ),
            showlegend=False,
            template="plotly_white"
        )

        recommended_hashtags = results['additional_analysis']['recommended_hashtags']
        references_needed = '참조 필요' if results['additional_analysis']['references_needed'] else '참조 필요 없음'

        return (
            f"이 게시물의 평균 점수: {user_score_average:.1f} ({user_star_rating})",
            fig,
            list(zip(results["categories"], results["scores"])),  # 카테고리와 점수를 데이터프레임 형식으로 반환
            "\n".join([f"{cat} - {score}/10\n설명: {exp}" for cat, score, exp in zip(results["categories"], results["scores"], results["explanations"])]),
            f"추천 해시태그: {', '.join(recommended_hashtags)}",
            references_needed
        )

# Gradio UI 구성
with gr.Blocks() as demo:
    gr.Markdown("# PPP - 소셜 미디어 게시물 인기도 예측기")
    gr.Markdown("당신의 게시물을 더 효과적으로 만들어 보세요!")

    # 입력 및 슬라이더
    with gr.Row():
        post_input = gr.Textbox(label="게시물을 입력하세요", placeholder="LinkedIn 게시물을 복사하여 붙여넣으세요...", lines=10)
        tone = gr.Slider(label="Tone (Official <-> Casual)", minimum=0, maximum=10, value=5)
        purpose = gr.Slider(label="Purpose (Informative <-> Motivational)", minimum=0, maximum=10, value=5)
        engagement = gr.Slider(label="Engagement (Engaging <-> Descriptive)", minimum=0, maximum=10, value=5)

    # 분석 버튼
    analyze_button = gr.Button("게시물 분석")

    # 결과 출력
    result_text = gr.Markdown()
    radar_chart = gr.Plot()
    category_names = gr.Dataframe(headers=["카테고리", "점수"], datatype=["str", "number"])
    recommendations = gr.Markdown()
    suggested_hashtags = gr.Markdown()
    references_info = gr.Markdown()

    # 버튼 클릭 시 함수 실행
    analyze_button.click(
        analyze_post,
        inputs=[post_input, tone, purpose, engagement],
        outputs=[result_text, radar_chart, category_names, recommendations, suggested_hashtags, references_info]
    )

# 실행
demo.launch()


==((====))==  Unsloth 2024.9.post4: Fast Gemma2 patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://9e81b8f26340cc9cc8.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
